# cyano-PS1 case study estimate scaling with equal area penalty

### Notebook helper function for pretty output

In [ ]:
from glotaran.utils.ipython import display_file

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

### Read data

In [ ]:
dataset = load_dataset("streakdata.ascii")
dataset

## Target analysis

Theoretical background from ``compartmental_models.pdf``

![model_sheme](target_scheme.jpg)


Concentration vector

$$c(t)= \begin{bmatrix} S(t) & B(t) & R_1(t) & R_2(t) & F(t)\end{bmatrix}^T$$

The concentration function correspond to

| ***Function name***    | ***S(t)***   | ***B(t)***   | ***R_1(t)***   | ***R_2(t)***   | ***F(t)***   |
|------------------------|--------------|--------------|----------------|----------------|--------------|
| **Name in the schema** | Soret        | Bulk         | Red1           | Red2           | Free         |


Differential equation

$$\dfrac{\mathrm{d}}{\mathrm{dt}}c(t) = \mathbf{K}c(t)+j(t)$$

Input function

$$j(t) = i(t) \begin{bmatrix} 1&0&0&0&0\end{bmatrix}^T$$

Transfer maxtrix $\mathbf{K}$

$$
\mathbf{K} = \begin{bmatrix} 
    -(k_{21} + k_{31} + k_{41} + k_{51}) &                          &                   &                  &       \\ 
    k_{21}                               & -(k_T + k_{32} + k_{42}) & k_{23}            & k_{24}           &       \\
    k_{31}                               &  k_{32}                  & -(k_{F} + k_{23}) &                  &       \\
    k_{41}                               &  k_{42}                  &                   & -(k_{F} + k_{24}) &      \\
    k_{51}                               &                          &                   &                  & k_{F} \\
    \end{bmatrix}
$$

- $k_T$: effective rate constantof Chl, T stands for photochemical Trapping of the excitation energy
- $k_F$: natural decay rate of free Chl, F stands for Fluorescence

### Reduced K-matrix

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
    k_{31} & k_{32} & k_{F}  &        &       \\
    k_{41} & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

### Reduced K-matrix ignoring input to Red1 and Red2

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
           & k_{32} & k_{F}  &        &       \\
           & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

With the rate relations:

- $k_{23} = a_1~k_{32}$
- $k_{24} = a_2~k_{42}$

### equal_area-target-model.yml

In [ ]:
display_file("models/equal_area-target-model.yml", syntax="yaml")

### equal_area-target-parameters.yml

In [ ]:
display_file("models/equal_area-target-parameters.yml", syntax="yml")

In [ ]:
target_model = load_model("models/equal_area-target-model.yml")
target_parameters = load_parameters("models/equal_area-target-parameters.yml")
target_model.validate(parameters=target_parameters)

In [ ]:
target_model

### Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    target_model,
    target_parameters,
    {"dataset1": dataset},
    non_negative_least_squares=True,
)
target_result = optimize(target_scheme)

In [ ]:
target_result.data["dataset1"]

### Result plots

In [ ]:
plot_overview(target_result.data["dataset1"], linlog=False)

In [ ]:
target_result.optimized_parameters